Step11
Date: May 13, 2024
Purpose: The purpose of this script is to do some formating and data scrubbing for the blast output generated by the previous step.
The previous step in the pipeline is: blast_miRBase_alignment_2024-5-13.sh (/home/administrator/Documents/Kaas/Venom_ncRNA_project/Scripts/blast/blast_miRBase_alignment_2024-5-13.sh)
The next step in this pipeline is: 3UTR_5UTR_exon_df_fusion_2024-5-13.ipynb (/home/administrator/Documents/Kaas/Venom_ncRNA_project/Scripts/Python/3UTR_5UTR_exon_df_fusion_2024-5-13.ipynb)

In [1]:
# Import needed packages
import pandas as pd
import numpy as np
import random

In [2]:
# Define function that can check and print if there are any query sequences between filtering steps:
def non_matching_queries(pre_filtering_df, post_filtering_df):
    # Create a data frame for only unique query sequences prior to filtering
    query_pre_filtering_df = pre_filtering_df['Query'].unique()
    # Create a data frame for only unique query sequneces after filtering
    query_post_filtering_df = post_filtering_df['Query'].unique()

    # Find queries that don't match between data sets
    non_matching_queries = set(query_pre_filtering_df) ^ set(query_post_filtering_df)

    # Print any queries that are not present after filtering or print that all queries are present
    if non_matching_queries:
        print('Queries sequences are missing after filtering. The non-matching queries are:')
        for query in non_matching_queries:
            print(query)
    else:
        print('No query sequences have been lost due to filtering.')


def find_best_blast_hit(row):
    closest_species = None

    for species in ['oha', 'pbv', 'aca', 'ami', 'cpi', 'xla', 'hsa', 'mmu', 'cfa', 'dre', 'cel', 'dme', 'dpu', 'ath', 'sly', 'zma']:
        if not pd.isnull(row[species]):
            closest_species = species
            break

    if closest_species is not None:
        # Split the blast hits by comma and choose one randomly
        blast_hits = row[closest_species].split(', ')
        selected_hit = random.choice(blast_hits)
        return f"{closest_species}-{selected_hit}"
    
    else:
        return np.nan

In [3]:
# Add another variable for the blast alignment so that it can be added to the final files
blast_alignment_df = pd.read_csv('/home/administrator/Documents/Kaas/Venom_ncRNA_project/Results/blast/miRBase/mature_mir-miRBase_alignment.tab', sep='\t')

# Add column names
blast_alignment_df.columns = ['Query', 'Subject', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 
                              'Blast.Gap.Openings', 'Blast.Query.Start', 'Blast.Query.End', 'Blast.Subject.Start', 'Blast.Subject.End', 
                              'E.value', 'Bit.Score']

# Create a new column miRNA.Start and miRNA.End
blast_alignment_df['miRNA.Start'] = blast_alignment_df['Query'].str.extract(r':(\d+)-').astype(int) 
blast_alignment_df['miRNA.End'] = blast_alignment_df['Query'].str.extract(r'-(\d+)').astype(int)
# Extract only the cluster name information:
blast_alignment_df['Query'] = blast_alignment_df['Query'].str.split('.').str[0]
# Create new column that is the length of the miRNA
blast_alignment_df['miRNA.Length'] = blast_alignment_df['miRNA.End'] - blast_alignment_df['miRNA.Start']
# Create a new column that is the absolute value of the Query Length plus 1
blast_alignment_df['Blast.Query.Length'] = abs(blast_alignment_df['Blast.Query.End'] - blast_alignment_df['Blast.Query.Start']) + 1
# Create a new column that is the absolute value of the Subject Length plus 1
blast_alignment_df['Blast.Subject.Length'] = abs(blast_alignment_df['Blast.Subject.End'] - blast_alignment_df['Blast.Subject.Start']) + 1
# Drop miRNA.End and miRNA.Start
blast_alignment_df.drop(['miRNA.Start', 'miRNA.End'], axis=1, inplace=True)

# Take the species abreviation out and give it it's own column
blast_alignment_df['Species.Abbreviation'] = blast_alignment_df['Subject'].str.split('-', n=1).str[0]
blast_alignment_df['Blast.Hits'] = blast_alignment_df['Subject'].str.split('-', n=1).str[1]

# New column order
column_order = ['Query', 'miRNA.Length', 'Subject', 'Species.Abbreviation', 'Blast.Hits', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 
                'Blast.Gap.Openings', 'Blast.Query.Start', 'Blast.Query.End', 'Blast.Query.Length', 'Blast.Subject.Start', 'Blast.Subject.End', 'Blast.Subject.Length',
                'E.value', 'Bit.Score' ] # Create new column order
blast_alignment_df = blast_alignment_df[column_order] # Reorder columns 

In [4]:
# Define a list of species abbreviations to keep
'''
Species are in he same order: 

        - Arabidopsis thaliana (ath)
		- Zea mays (zma)
		- Caenorhabditis elegans (cel)
		- Daphnia pulex (dpu)
		- Homo sapiens (hsa)
		- Anolis carolinensis (aca)
		- Danio rerio (dre)
		- Drosophila melanogaster (dme)
		- Mus musculus (mmu)
		- Canis familiaris (cfa)
		- Solanum lycopersicum (sly)
        - Ophiophagus hannah (oha)
		- Python bivittatus (pbv)
		- Alligator mississippiensis (ami)
		- Chrysemys picta (cpi)
		- Xenopus laevis (xla)
'''

species_to_keep = [
    'ath', 'zma', 'cel', 'dpu', 'hsa', 'aca', 'dre', 'dme', 'mmu', 'cfa', 'sly', 
    'oha', 'pbv', 'ami', 'cpi', 'xla'
]

# Create a boolean mask indicating which rows to keep
mask = blast_alignment_df['Species.Abbreviation'].isin(species_to_keep)

# Apply filter
taxa_filtered_df = blast_alignment_df[mask]

# Check what queries, if any, have gone missing.
non_matching_queries(blast_alignment_df, taxa_filtered_df)
'''
Result:
No query sequences have been lost due to filtering.
'''

No query sequences have been lost due to filtering.


'\nResult:\nNo query sequences have been lost due to filtering.\n'

In [6]:
# E-value threshold
e_value_threshold = 0.001
# Filter out E.values that are too low.
e_value_filtered_df = taxa_filtered_df[taxa_filtered_df['E.value'] < e_value_threshold]

# Check what queries, if any, have gone missing.
non_matching_queries(taxa_filtered_df, e_value_filtered_df)
'''
Queries sequences are missing after filtering. The non-matching queries are:
Cluster_1858
Cluster_1718
Cluster_182
Cluster_988
Cluster_186
Cluster_1226
Cluster_807
Cluster_1337
Cluster_341
Cluster_1326
Cluster_1578
Cluster_1395
Cluster_196
Cluster_1292
Cluster_1640
Cluster_853
Cluster_1037
Cluster_1428
Cluster_919
Cluster_1666
Cluster_659
'''

# Create a data frame for clusters that were lost from filtering by e-value
# Create a list of lost clusters
lost_clusters = [
    'Cluster_1858',
    'Cluster_1718',
    'Cluster_182',
    'Cluster_988',
    'Cluster_186',
    'Cluster_1226',
    'Cluster_807',
    'Cluster_1337',
    'Cluster_341',
    'Cluster_1326',
    'Cluster_1578',
    'Cluster_1395',
    'Cluster_196',
    'Cluster_1292',
    'Cluster_1640',
    'Cluster_853',
    'Cluster_1037',
    'Cluster_1428',
    'Cluster_919',
    'Cluster_1666',
    'Cluster_659'
]

# Get lost clusters and put them a data frame
lost_clusters_e_value_df = taxa_filtered_df[taxa_filtered_df['Query'].isin(lost_clusters)]

# Group by 'Query' (cluster) and find the row with the lowest E-value for each group
best_hits_for_lost_clusters_df = lost_clusters_e_value_df.loc[lost_clusters_e_value_df.groupby('Query')['E.value'].idxmin()]

best_hits_for_lost_clusters_df.reset_index(inplace=True)

Queries sequences are missing after filtering. The non-matching queries are:
Cluster_1858
Cluster_1718
Cluster_182
Cluster_988
Cluster_186
Cluster_1226
Cluster_807
Cluster_1337
Cluster_341
Cluster_1326
Cluster_1578
Cluster_1395
Cluster_196
Cluster_1292
Cluster_1640
Cluster_853
Cluster_1037
Cluster_1428
Cluster_919
Cluster_1666
Cluster_659


In [7]:
# Create a pivoted data frame for the first missing clusters data frame
pivot_best_lost_clusters_df = pd.pivot_table(
    best_hits_for_lost_clusters_df,
    index=['Query', 'miRNA.Length', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 'Blast.Gap.Openings', 
           'Blast.Query.Start', 'Blast.Query.End', 'Blast.Query.Length', 'Blast.Subject.Start', 'Blast.Subject.End', 'Blast.Subject.Length', 
           'E.value', 'Bit.Score'],
    columns='Species.Abbreviation',
    values='Blast.Hits',
    aggfunc=lambda x: ', '.join(x)
)
#print(pivot_best_lost_clusters_df)

# Reset index
pivot_best_lost_clusters_df.reset_index(inplace=True)

# Add missing columns so the function works
pivot_best_lost_clusters_df['pbv'] = np.nan
pivot_best_lost_clusters_df['cpi'] = np.nan
pivot_best_lost_clusters_df['dre'] = np.nan
pivot_best_lost_clusters_df['cel'] = np.nan
pivot_best_lost_clusters_df['dpu'] = np.nan
pivot_best_lost_clusters_df['sly'] = np.nan


# Set to new df
best_lost_matches_df1 = pivot_best_lost_clusters_df
best_lost_matches_df2 = best_lost_matches_df1

# Here I run the above function to create a new column
best_lost_matches_df1['Best.miRNA.Blast.Hits'] = best_lost_matches_df1.apply(find_best_blast_hit, axis=1)

# Create another column for just regular names
best_lost_matches_df1['Base.Putative.miRNA.Name'] = best_lost_matches_df1['Best.miRNA.Blast.Hits'].str.split('-', n=1).str[1]

# Create another column to classify what whether the miRNA is probably the same or not.
best_lost_matches_df1['miRNA.Identity.Type'] = 'De-Novo'

In [8]:
# Further filter down to so that the percent identity is exactly the same
percent_id_df = e_value_filtered_df[e_value_filtered_df['Blast.Percent.Identity'] == 100]

# Check what queries are missing after filtering for percent identity
non_matching_queries(e_value_filtered_df, percent_id_df)
'''
Result:
No query sequences have been lost due to filtering.
'''

No query sequences have been lost due to filtering.


'\nResult:\nNo query sequences have been lost due to filtering.\n'

In [11]:
# Create a new dataframe where the alignment length and the query end match exactly so that there isn't any chance of an alignment being non-identical due to the alignment starting at a different place than the miRNA
length_match_df = percent_id_df[percent_id_df['Blast.Alignment.Length'] == percent_id_df['miRNA.Length']]

# Further filter down based on whether the miRNA.Length equals the length of the subject sequence
miRNA_length_match_df = length_match_df[length_match_df['miRNA.Length'] == length_match_df['Blast.Subject.Length']]
print('Are length_match_df and miRNA_length_match_df the same?', miRNA_length_match_df.equals(length_match_df))

# Check if any queries were lost from filtering
non_matching_queries(percent_id_df, miRNA_length_match_df)
'''
Queries sequences are missing after filtering. The non-matching queries are:
Cluster_1863
Cluster_866
Cluster_1339
Cluster_856
Cluster_1692
'''
# Create a list of filtered out clusters
non_matching_length = [
    'Cluster_1863',
    'Cluster_866',
    'Cluster_1339',
    'Cluster_856',
    'Cluster_1692'
]

# Check if create dataframe of lost clusters
non_matching_length_df = percent_id_df[percent_id_df['Query'].isin(non_matching_length)]

# Further filter down based on whether the length of the subject sequence equals either it's start or end.
# This will take out sequences that have an extra nucleotide on either end
best_length_match_df = miRNA_length_match_df[
    (miRNA_length_match_df['miRNA.Length'] == miRNA_length_match_df['Blast.Subject.End']) |
    (miRNA_length_match_df['miRNA.Length'] == miRNA_length_match_df['Blast.Subject.Start'])
]

# Check if any queries were lost after filtering for homolog length identity
non_matching_queries(miRNA_length_match_df, best_length_match_df)
'''
Result:
Queries sequences are missing after filtering. The non-matching queries are:
Cluster_1772
Cluster_822
'''

# Create a new data frame of lost clusters that were taken
# Create a list of Clusters that were filtered out
filtered_out_clusters = [
    'Cluster_1772',
    'Cluster_822'
]

# Get lost clusters and put them a dataframe
length_filtered_lost_clusters_df = miRNA_length_match_df[miRNA_length_match_df['Query'].isin(filtered_out_clusters)]

Are length_match_df and miRNA_length_match_df the same? True
Queries sequences are missing after filtering. The non-matching queries are:
Cluster_1863
Cluster_866
Cluster_1339
Cluster_856
Cluster_1692
Queries sequences are missing after filtering. The non-matching queries are:
Cluster_1772
Cluster_822


In [12]:
# Group by 'Query' (cluster) and find the row with the highest alignment length
non_matching_length_df = non_matching_length_df.loc[non_matching_length_df.groupby('Query')['Blast.Alignment.Length'].idxmax()]


# Create a new data frame that is a pivoted dataframe of what was lost after filter for length
pivot_non_matching_length_df = pd.pivot_table(
    non_matching_length_df,
    index=['Query', 'miRNA.Length', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 'Blast.Gap.Openings', 
           'Blast.Query.Start', 'Blast.Query.End', 'Blast.Query.Length', 'Blast.Subject.Start', 'Blast.Subject.End', 'Blast.Subject.Length', 
           'E.value', 'Bit.Score'],
    columns='Species.Abbreviation',
    values='Blast.Hits',
    aggfunc=lambda x: ', '.join(x)
)

# Print to check if it worked
#print(pivot_non_matching_length_df)

# Assign to new data frame
best_missing_matches_by_length_df = pivot_non_matching_length_df

# Reset index
best_missing_matches_by_length_df.reset_index(inplace=True)

# Add missing columns
best_missing_matches_by_length_df['ami'] = np.nan
best_missing_matches_by_length_df['cpi'] = np.nan
best_missing_matches_by_length_df['sly'] = np.nan
best_missing_matches_by_length_df['cfa'] = np.nan
best_missing_matches_by_length_df['mmu'] = np.nan
best_missing_matches_by_length_df['dme'] = np.nan
best_missing_matches_by_length_df['dre'] = np.nan
best_missing_matches_by_length_df['aca'] = np.nan
best_missing_matches_by_length_df['hsa'] = np.nan
best_missing_matches_by_length_df['dpu'] = np.nan
best_missing_matches_by_length_df['cel'] = np.nan
best_missing_matches_by_length_df['zma'] = np.nan
best_missing_matches_by_length_df['ath'] = np.nan

# Use function to create best hits column
best_missing_matches_by_length_df['Best.miRNA.Blast.Hits'] = best_missing_matches_by_length_df.apply(find_best_blast_hit, axis=1)

# Add putative miRNA names
best_missing_matches_by_length_df['Base.Putative.miRNA.Name'] = best_missing_matches_by_length_df['Best.miRNA.Blast.Hits'].str.split('-', n=1).str[1]

# Create another column to classify the likelihood of the name being correct
best_missing_matches_by_length_df['miRNA.Identity.Type'] = 'Potential-Identity'
print(best_missing_matches_by_length_df)

Species.Abbreviation         Query  miRNA.Length  Blast.Percent.Identity  \
0                     Cluster_1339            22                   100.0   
1                     Cluster_1692            22                   100.0   
2                     Cluster_1863            22                   100.0   
3                      Cluster_856            22                   100.0   
4                      Cluster_866            23                   100.0   

Species.Abbreviation  Blast.Alignment.Length  Blast.Mismatches  \
0                                         21                 0   
1                                         21                 0   
2                                         21                 0   
3                                         21                 0   
4                                         22                 0   

Species.Abbreviation  Blast.Gap.Openings  Blast.Query.Start  Blast.Query.End  \
0                                      0                  1       

In [13]:
# Create a new data frame that is a pivoted dataframe of what was lost after filter for length
pivot_length_filtered_lost_clusters_df = pd.pivot_table(
    length_filtered_lost_clusters_df,
    index=['Query', 'miRNA.Length', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 'Blast.Gap.Openings', 
           'Blast.Query.Start', 'Blast.Query.End', 'Blast.Query.Length', 'Blast.Subject.Start', 'Blast.Subject.End', 'Blast.Subject.Length', 
           'E.value', 'Bit.Score'],
    columns='Species.Abbreviation',
    values='Blast.Hits',
    aggfunc=lambda x: ', '.join(x)
)

# Reset index to flatten the multi-index
pivot_length_filtered_lost_clusters_df.reset_index(inplace=True)

# Add missing columns so the function works
pivot_length_filtered_lost_clusters_df['ath'] = np.nan
pivot_length_filtered_lost_clusters_df['cfa'] = np.nan
pivot_length_filtered_lost_clusters_df['ami'] = np.nan
pivot_length_filtered_lost_clusters_df['ath'] = np.nan
pivot_length_filtered_lost_clusters_df['cel'] = np.nan
pivot_length_filtered_lost_clusters_df['dme'] = np.nan
pivot_length_filtered_lost_clusters_df['dpu'] = np.nan
pivot_length_filtered_lost_clusters_df['hsa'] = np.nan
pivot_length_filtered_lost_clusters_df['sly'] = np.nan
pivot_length_filtered_lost_clusters_df['zma'] = np.nan



# Here I create a dataframe to equal the pivoted current one so I can do things to it with altering the old one
best_lost_matches_df2 = pivot_length_filtered_lost_clusters_df

# Here I run the above functions to creat new columns that have good matches for each miRNA
best_lost_matches_df2['Best.miRNA.Blast.Hits'] = best_lost_matches_df2.apply(find_best_blast_hit, axis=1)

# Create another column for just regular names without the species attachment
best_lost_matches_df2['Base.Putative.miRNA.Name'] = best_lost_matches_df2['Best.miRNA.Blast.Hits'].str.split('-', n=1).str[1]

# Create another column to classify what whether the miRNA is probably the same or not.
best_lost_matches_df2['miRNA.Identity.Type'] = 'Probable-Identity'

# Check if it worked
#print(best_lost_matches_df2['miRNA.Name'].to_string(index=False))

In [14]:
# Create a new data frame that is a pivoted data frame of the the most filtered data frame so I can figure out what the best hits are
pivot_best_length_match_df = pd.pivot_table(
    best_length_match_df,
    index=['Query', 'miRNA.Length', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 'Blast.Gap.Openings', 
           'Blast.Query.Start', 'Blast.Query.End', 'Blast.Query.Length', 'Blast.Subject.Start', 'Blast.Subject.End', 'Blast.Subject.Length', 
           'E.value', 'Bit.Score'],
    columns='Species.Abbreviation',
    values='Blast.Hits',
    aggfunc=lambda x: ', '.join(x)
)

# Reset index to flatten the multi-index
pivot_best_length_match_df.reset_index(inplace=True)

# Here I create a dataframe to equal the pivoted current one so I can do things to it with altering the old one
best_matches_by_species_df = pivot_best_length_match_df

# Here I run the above functions to creat new columns that have good matches for each miRNA
best_matches_by_species_df['Best.miRNA.Blast.Hits'] = best_matches_by_species_df.apply(find_best_blast_hit, axis=1)

# Create another column for just regular names without the species attachment
best_matches_by_species_df['Base.Putative.miRNA.Name'] = best_matches_by_species_df['Best.miRNA.Blast.Hits'].str.split('-', n=1).str[1]

'''
After I pivoted the dataframe and added the new columns I noticed that some of the new columns have multiple entries in them. Since all of these entries at this
point are equally good statistically, I decided to just have it decided randomly.
'''

# Create another column to classify what whether the miRNA is probably the same or not.
best_matches_by_species_df['miRNA.Identity.Type'] = 'Very-Probable-Identity'

# Check if it worked
#print(best_matches_by_species_df['miRNA.Name'].to_string(index=False))

In [16]:
# Concatenate from top to bottom
final_df = pd.concat([best_matches_by_species_df, best_lost_matches_df2, best_lost_matches_df1, best_missing_matches_by_length_df], axis=0, ignore_index=True)
final_df2 = pd.concat([best_matches_by_species_df, best_lost_matches_df2, best_lost_matches_df1, best_missing_matches_by_length_df], axis=0, ignore_index=True)

# Reorder dataframe
column_order = ['Query','Best.miRNA.Blast.Hits', 'Base.Putative.miRNA.Name', 'miRNA.Identity.Type',
                'oha', 'pbv', 'aca', 'ami', 'cpi', 'xla', 'hsa', 'mmu', 'cfa', 'dre', 'cel', 'dme', 'dpu', 'ath', 'sly', 'zma', 
                'miRNA.Length', 'Blast.Percent.Identity', 'Blast.Alignment.Length', 'Blast.Mismatches', 
                'Blast.Gap.Openings', 'Blast.Query.Start', 'Blast.Query.End', 'Blast.Query.Length', 'Blast.Subject.Start', 'Blast.Subject.End', 'Blast.Subject.Length',
                'E.value', 'Bit.Score' ]

# Apply column order
final_df = final_df[column_order]

# Take out the species abbreviation again
final_df['Species.Abbreviation'] = final_df['Best.miRNA.Blast.Hits'].str.split('-', n=1).str[0]

def remove_other_species_hits(final_df):
    # Define the hierarchy of species abbreviations to prioritize
    species_hierarchy = ['oha', 'pbv', 'aca', 'ami', 'cpi', 'xla', 'hsa', 'mmu', 'cfa', 'dre', 'cel', 'dme', 'dpu', 'ath', 'sly', 'zma']
    
    # Initialize dictionaries to store the highest priority species and hits for each query
    query_highest_priority_species = {}
    query_highest_priority_hit = {}
    
    # Iterate through rows in the DataFrame to determine highest priority species and hits for each query
    for index, row in final_df.iterrows():
        query = row['Query']
        species = row['Species.Abbreviation']
        
        # If this is the first encounter of the query, initialize its highest priority species and hit
        if query not in query_highest_priority_species:
            query_highest_priority_species[query] = species
            query_highest_priority_hit[query] = index
        else:
            # Check if the current species has higher priority than the stored highest priority species for this query
            if species_hierarchy.index(species) < species_hierarchy.index(query_highest_priority_species[query]):
                # Update highest priority species and hit for this query
                query_highest_priority_species[query] = species
                query_highest_priority_hit[query] = index
    
    # Initialize a list to store rows to be dropped
    rows_to_drop = []
    
    # Iterate through rows in the DataFrame to drop lower priority species hits
    for index, row in final_df.iterrows():
        query = row['Query']
        species = row['Species.Abbreviation']
        
        # Check if the species for this row is not the highest priority for its query
        if index != query_highest_priority_hit[query]:
            # Mark the row to be dropped
            rows_to_drop.append(index)
    
    # Drop rows marked for deletion
    final_df = final_df.drop(rows_to_drop)
    
    return final_df

# Check if any Queries were lost
non_matching_queries(blast_alignment_df, final_df)

# Call the function to remove other species hits
final_df = remove_other_species_hits(final_df)

# Drop the species abbreviation column
final_df.drop(columns = ['Species.Abbreviation'], inplace = True)

# Check if any Queries were lost
non_matching_queries(final_df2, final_df)
non_matching_queries(blast_alignment_df, final_df)

# Save to csv
final_df.to_csv('/home/administrator/Documents/Kaas/Venom_ncRNA_project/Results/blast/miRBase/miRNA_formated_alignment.tsv', sep='\t')

No query sequences have been lost due to filtering.
No query sequences have been lost due to filtering.
No query sequences have been lost due to filtering.
